In [1]:
!pip install langchain-ollama langchain langchain-community Pillow faiss-cpu

In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="qwen:7b", temperature=0.7)

messages = [
    ("human", "你好呀"),
]

for chunk in model.stream(messages):
    print(chunk.content, end='', flush=True)


你好！有什么问题或者需要帮助的吗？

In [3]:
!conda env list

# conda environments:
#
base                     D:\app\anaconda3
coursedesign          *  D:\app\anaconda3\envs\coursedesign
latex                    D:\app\anaconda3\envs\latex
test                     D:\app\anaconda3\envs\test



In [4]:
# 初始化 Ollama 模型和嵌入

from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

llm = ChatOllama(model="qwen:7b")
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# 准备文档
text = """
Datawhale 是一个专注于数据科学与 AI 领域的开源组织，汇集了众多领域院校和知名企业的优秀学习者，聚合了一群有开源精神和探索精神的团队成员。
Datawhale 以" for the learner，和学习者一起成长"为愿景，鼓励真实地展现自我、开放包容、互信互助、敢于试错和勇于担当。
同时 Datawhale 用开源的理念去探索开源内容、开源学习和开源方案，赋能人才培养，助力人才成长，建立起人与人，人与知识，人与企业和人与未来的联结。
如果你想在Datawhale开源社区发起一个开源项目，请详细阅读Datawhale开源项目指南[https://github.com/datawhalechina/DOPMC/blob/main/GUIDE.md]
"""

# with open('斗破苍穹.txt', 'r', encoding='utf-8') as file:
#     text = file.read()

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = text_splitter.split_text(text)

print(chunks)
# 创建向量存储
vectorstore = FAISS.from_texts(chunks, embeddings)
retriever = vectorstore.as_retriever()

#region 查看vectorstore的信息
"""
texts = chunks

vectorstore = FAISS.from_texts(texts, embeddings)


print("向量数量:", vectorstore.index.ntotal)
print("文本片段数量:", len(texts))


for i, text in enumerate(texts):
    print(f"文本片段 {i}: {text}")
    print(f"向量 {i}: {vectorstore.index.reconstruct(i)}")

vector = vectorstore.index.reconstruct(0)
print("向量维度:", len(vector))
"""
#endregion


['Datawhale 是一个专注于数据科学与 AI 领域的开源组织，汇集了众多领域院校和知名企业的优秀学习者，聚合了一群有开源精神和探索精神的团队成员。', 'Datawhale 以" for the learner，和学习者一起成长"为愿景，鼓励真实地展现自我、开放包容、互信互助、敢于试错和勇于担当。', '同时 Datawhale 用开源的理念去探索开源内容、开源学习和开源方案，赋能人才培养，助力人才成长，建立起人与人，人与知识，人与企业和人与未来的联结。', '如果你想在Datawhale开源社区发起一个开源项目，请详细阅读Datawhale开源项目指南[https://github.com/datawhalechina/DOPMC/blob/main/G', 'na/DOPMC/blob/main/GUIDE.md]']


'\ntexts = chunks\n\nvectorstore = FAISS.from_texts(texts, embeddings)\n\n\nprint("向量数量:", vectorstore.index.ntotal)\nprint("文本片段数量:", len(texts))\n\n\nfor i, text in enumerate(texts):\n    print(f"文本片段 {i}: {text}")\n    print(f"向量 {i}: {vectorstore.index.reconstruct(i)}")\n\nvector = vectorstore.index.reconstruct(0)\nprint("向量维度:", len(vector))\n'

In [5]:
# 创建提示模板
template = """只能使用下列内容回答问题:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='只能使用下列内容回答问题:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})]


In [6]:
from langchain_core.runnables import Runnable

class DebugPrompt(Runnable):
    def invoke(self, inputs, context):
        print("最终传递给 llm 的提示:", inputs)
        return inputs

# 创建调试步骤
debug_prompt = DebugPrompt()

# 创建检索-问答链，并插入调试步骤
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | debug_prompt
    | llm
)

# 使用链回答问题
question = "datawhale 的愿景是什么"
response = chain.invoke(question)


最终传递给 llm 的提示: messages=[HumanMessage(content='只能使用下列内容回答问题:\n[Document(id=\'254c0a5b-d281-4874-8f0e-09dbe328bc54\', metadata={}, page_content=\'如果你想在Datawhale开源社区发起一个开源项目，请详细阅读Datawhale开源项目指南[https://github.com/datawhalechina/DOPMC/blob/main/G\'), Document(id=\'efe8db3b-5865-442b-a101-c3aada934487\', metadata={}, page_content=\'同时 Datawhale 用开源的理念去探索开源内容、开源学习和开源方案，赋能人才培养，助力人才成长，建立起人与人，人与知识，人与企业和人与未来的联结。\'), Document(id=\'5fc6519e-6853-4937-b097-0d84d13e257a\', metadata={}, page_content=\'Datawhale 是一个专注于数据科学与 AI 领域的开源组织，汇集了众多领域院校和知名企业的优秀学习者，聚合了一群有开源精神和探索精神的团队成员。\'), Document(id=\'d3b13873-08ca-4b94-99f8-10aac07856f5\', metadata={}, page_content=\'Datawhale 以" for the learner，和学习者一起成长"为愿景，鼓励真实地展现自我、开放包容、互信互助、敢于试错和勇于担当。\')]\n\nQuestion: datawhale 的愿景是什么\n', additional_kwargs={}, response_metadata={})]


In [7]:
print(response.content)

datawhale的愿景是'for the learner，和学习者一起成长'，鼓励真实展现自我、开放包容等价值观。
